# AI Agents Crash Course - Part 2

# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

# Load Required Python Modules and Libraries

In [1]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

In [ ]:
# Uncomment in order to enable litellm debugging for better error diagnostics
#
# import litellm
# litellm._turn_on_debug()

# Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [2]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')

# Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [4]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    model="ollama/llama3.2:1b",
    base_url=OLLAMA_API_BASE
)

# Verify LLM Configuration

## Universal LLM Connection Test

In [5]:
import requests
import os

def test_llm_connection():
    """Test LLM connection regardless of provider"""
    
    print("=== LLM Configuration Analysis ===")
    
    # Analyze LLM configuration
    model_name = getattr(llm, 'model', 'Unknown')
    base_url = getattr(llm, 'base_url', None)
    api_key_set = bool(getattr(llm, 'api_key', None))
    
    print(f"Model: {model_name}")
    print(f"Base URL: {base_url if base_url else 'Default (provider-specific)'}")
    print(f"API Key Set: {'Yes' if api_key_set else 'No'}")
    
    # Determine provider type
    provider = "unknown"
    if "ollama" in model_name.lower():
        provider = "ollama"
    elif "gpt" in model_name.lower() or "openai" in model_name.lower():
        provider = "openai"
    elif "claude" in model_name.lower() or "anthropic" in model_name.lower():
        provider = "anthropic"
    elif "gemini" in model_name.lower() or "google" in model_name.lower():
        provider = "google"
    
    print(f"Detected Provider: {provider}")
    
    # Provider-specific connection tests
    print(f"\n=== {provider.title()} Connection Test ===")
    
    if provider == "ollama" and base_url:
        try:
            # Test Ollama server availability
            test_url = f"{base_url}/api/tags"
            response = requests.get(test_url, timeout=5)
            print(f"Ollama Server Status: {response.status_code}")
            if response.status_code == 200:
                models = response.json().get('models', [])
                print(f"Available Models: {len(models)} found")
                # Check if our specific model is available
                model_available = any(model_name.replace('ollama/', '') in str(model) for model in models)
                print(f"Target Model Available: {'Yes' if model_available else 'No'}")
            else:
                print(f"Ollama server responded with status: {response.status_code}")
        except Exception as e:
            print(f"❌ Ollama server connection failed: {e}")
    
    elif provider == "openai":
        print("OpenAI connection test (API key validation happens during LLM call)")
        api_key_env = os.getenv('OPENAI_API_KEY')
        print(f"OPENAI_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "anthropic":
        print("Anthropic connection test")
        api_key_env = os.getenv('ANTHROPIC_API_KEY')
        print(f"ANTHROPIC_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    elif provider == "google":
        print("Google AI connection test")
        api_key_env = os.getenv('GOOGLE_API_KEY')
        print(f"GOOGLE_API_KEY environment variable: {'Set' if api_key_env else 'Not set'}")
    
    else:
        print("Generic provider - will test with LLM call only")
    
    # Universal LLM functionality test
    print(f"\n=== LLM Functionality Test ===")
    try:
        test_response = llm.call([{"role": "user", "content": "Respond with exactly: 'Test successful'"}])
        print("✅ LLM call successful!")
        print(f"Response type: {type(test_response)}")
        
        # Try to extract response content
        if hasattr(test_response, 'content'):
            print(f"Response content: {test_response.content[:100]}...")
        elif isinstance(test_response, str):
            print(f"Response: {test_response[:100]}...")
        else:
            print(f"Response: {str(test_response)[:100]}...")
            
    except Exception as e:
        print(f"❌ LLM call failed: {e}")
        print(f"Error type: {type(e).__name__}")
        
        # Provide specific troubleshooting tips based on error
        error_str = str(e).lower()
        if "connection" in error_str or "timeout" in error_str:
            print("💡 Tip: Check network connection and base_url configuration")
        elif "api_key" in error_str or "authentication" in error_str or "unauthorized" in error_str:
            print("💡 Tip: Check API key configuration and permissions")
        elif "model" in error_str or "not found" in error_str:
            print("💡 Tip: Verify model name and availability")

# Run the comprehensive test
test_llm_connection()

=== LLM Configuration Analysis ===
Model: ollama/llama3.2:1b
Base URL: http://host.docker.internal:11434
API Key Set: No
Detected Provider: ollama

=== Ollama Connection Test ===
Ollama Server Status: 200
Available Models: 6 found
Target Model Available: Yes

=== LLM Functionality Test ===
✅ LLM call successful!
Response type: <class 'str'>
Response: Test successful....


## Environment Variables Check for All LLM Providers

In [6]:
def check_environment_variables():
    """Check environment variables for all major LLM providers"""
    
    print("=== Environment Variables Status ===")
    
    # Common LLM provider environment variables
    env_vars = {
        "OpenAI": ["OPENAI_API_KEY", "OPENAI_BASE_URL"],
        "Anthropic": ["ANTHROPIC_API_KEY"],
        "Google": ["GOOGLE_API_KEY", "GOOGLE_APPLICATION_CREDENTIALS"],
        "Cohere": ["COHERE_API_KEY"],
        "Hugging Face": ["HUGGINGFACE_API_KEY", "HF_TOKEN"],
        "Ollama": ["OLLAMA_API_BASE", "OLLAMA_HOST"],
        "Azure OpenAI": ["AZURE_OPENAI_API_KEY", "AZURE_OPENAI_ENDPOINT"],
        "AWS Bedrock": ["AWS_ACCESS_KEY_ID", "AWS_SECRET_ACCESS_KEY", "AWS_REGION"],
        "Together AI": ["TOGETHER_API_KEY"],
        "Replicate": ["REPLICATE_API_TOKEN"],
        "Perplexity": ["PERPLEXITYAI_API_KEY"],
        "Groq": ["GROQ_API_KEY"]
    }
    
    found_providers = []
    
    for provider, vars_list in env_vars.items():
        provider_vars = {}
        has_any_var = False
        
        for var in vars_list:
            value = os.getenv(var)
            if value:
                provider_vars[var] = "✅ Set"
                has_any_var = True
            else:
                provider_vars[var] = "❌ Not set"
        
        if has_any_var:
            found_providers.append(provider)
            print(f"\n{provider}:")
            for var, status in provider_vars.items():
                print(f"  {var}: {status}")
    
    if not found_providers:
        print("No LLM provider environment variables found.")
        print("Make sure to set the appropriate API keys for your chosen provider.")
    else:
        print(f"\nConfigured providers: {', '.join(found_providers)}")

# Run environment check
check_environment_variables()

=== Environment Variables Status ===

OpenAI:
  OPENAI_API_KEY: ✅ Set
  OPENAI_BASE_URL: ❌ Not set

Ollama:
  OLLAMA_API_BASE: ✅ Set
  OLLAMA_HOST: ✅ Set

Configured providers: OpenAI, Ollama


# Building multi-agent systems - Building Modular Crews

In [22]:
# Original import and usage - now fixed!
from research_crew import ResearchCrew

research_crew = ResearchCrew()

result = research_crew.crew().kickoff(inputs={"topic": "The impact of AI on job markets"})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 552c6270-4174-4fb4-8789-82f755546ed4                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Task: Use the SerperDevTool to find the most relevant and recent data on The impact of AI on job markets.      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/home/vscode/.local/lib/python3.11/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for Jupyter 
support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Internet Researcher                                                                                     │
│                                                                                                                 │
│  Thought: I need to research the impact of AI on job markets to gather relevant and recent data, insights, and  │
│  source references.                                                                                             │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"impact of AI on job markets 2023 report\"}"                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"PwC AI jobs economic growth report\"}"                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': 'PwC AI jobs economic growth report', 'type': 'search', 'num': 10, 'engine':        │
│  'google'}, 'organic': [{'title': 'The Fearless Future: 2025 Global AI Jobs Barometer - PwC', 'link':           │
│  'https://www.pwc.com/gx/en/issues/artificial-intelligence/ai-jobs-barometer.html', 'snippet': 'Industries      │
│  more exposed to AI have 3x higher growth in revenue per employee · Wages are rising 2x faster in the most      │
│  AI-exposed industries.', 'position': 1}, {'title': 'PwC 2025 Global AI Jobs Barometer', 'link':                │
│  'https://www.pwc.com/gx/en/news-room/press-releases/2025/ai-linked-to-a-fourfold-increase-in-productivity-gro  │
│  wth.html', 'snippet': 'AI linked to a fourfold increase in productivity growth and 56% wage premium, while     │
│  jobs grow even in the most easily automated roles · Job ...', 'position': 2}, {'title': '[PDF] 2025 Global AI  │
│  Jobs Barometer - US Analysis - PwC', 'link':                                                                   │
│  'https://www.pwc.com/gx/en/issues/artificial-intelligence/job-barometer/aijb-2025-united-states-analysis.pdf'  │
│  , 'snippet': 'Skills sought by employers for AI-exposed jobs are changing 66% faster than for other jobs – up  │
│  from 25% last year. AI is associated with gentler growth – but ...', 'position': 3}, {'title': '5 takeaways    │
│  from the 2025 AI Jobs Barometer - PwC', 'link':                                                                │
│  'https://www.pwc.com/us/en/tech-effect/ai-analytics/ai-jobs-barometer.html', 'snippet': 'AI-exposed US         │
│  industries saw revenue per employee jump 27% — over 3x the growth in less AI-ready sectors. US workers with    │
│  advanced AI skills ...', 'position': 4}, {'title': 'PwC releases 2025 Global AI Jobs Barometer', 'link':       │
│  'https://www.pwchk.com/en/press-room/press-releases/pr-130625.html', 'snippet': 'AI linked to a fourfold       │
│  increase in productivity growth and 56% wage premium globally; demand for roles requiring AI-related skills    │
│  increased in ...', 'position': 5}, {'title': '[PDF] The Fearless Future: 2025 Global AI Jobs Barometer -       │
│  PwC', 'link': 'https://www.pwc.com/gx/en/issues/artificial-intelligence/job-barometer/2025/report.pdf', ...    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Fact-Checking Specialist                                                                                │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ### Research Report: Impact of AI on Job Markets                                                               │
│                                                                                                                 │
│  #### Introduction                                                                                              │
│  The rise of artificial intelligence (AI) is reshaping labor markets globally. This report synthesizes          │
│  findings from recent studies that assess the impact of AI on employment, focusing on job displacement,         │
│  creation, and transformation.                                                                                  │
│                                                                                                                 │
│  #### Key Insights                                                                                              │
│                                                                                                                 │
│  1. **Job Displacement vs. Job Creation**                                                                       │
│     - The **World Economic Forum’s “Future of Jobs Report 2023”** estimates that by 2027, AI and automation     │
│  could displace **83 million jobs**, while also creating **69 million new jobs** globally. This indicates a     │
│  net loss in employment overall, which emphasizes the transition that workers would face (Source: [World        │
│  Economic Forum](https://www.weforum.org/publications/the-future-of-jobs-report-2023/)).                        │
│                                                                                                                 │
│  2. **Automation Potential**                                                                                    │
│     - A report by **Goldman Sachs** posits that **300 million full-time jobs** worldwide are at risk from AI    │
│  automation, underscoring generative AI's potential to disrupt a wide range of industries (Source: [Goldman     │
│  Sachs Report](https://www.iedconline.org/clientuploads/EDRP%20Logos/AI_Impact_on_Labor_Markets.pdf)).          │
│                                                                                                                 │
│  3. **Industry-Specific Impacts**                                                                               │
│     - The **Bureau of Labor Statistics (BLS)** anticipates a **17.9%** increase in employment for software      │
│  developers from 2023 to 2033, which greatly outpaces the average growth rate of **4%** across all              │
│  occupations, highlighting the increasing demand for AI-related skills (Source: [BLS                            │
│  Report](https://www.bls.gov/opub/ted/2025/ai-impacts-in-bls-employment-projections.htm)).                      │
│     - A **McKinsey report** indicates that **30% of current U.S. jobs** could be automated by 2030, suggesting  │
│  profound changes in the job landscape due to AI. This indicates a serious understanding of the shifts          │
│  required in the workforce (Source: [Forbes                                                                     │
│  Article](https://www.forbes.com/sites/jackkelly/2025/04/25/the-jobs-that-will-fall-first-as-ai-takes-over-the  │
│  -workplace/)).                                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 043aba13-6118-46e1-8b0f-a26e4e8299df                                                                     │
│  Agent: Fact-Checking Specialist                                                                                │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 552c6270-4174-4fb4-8789-82f755546ed4                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ### Research Report: Impact of AI on Job Markets                                                 │
│                                                                                                                 │
│  #### Introduction                                                                                              │
│  The rise of artificial intelligence (AI) is reshaping labor markets globally. This report synthesizes          │
│  findings from recent studies that assess the impact of AI on employment, focusing on job displacement,         │
│  creation, and transformation.                                                                                  │
│                                                                                                                 │
│  #### Key Insights                                                                                              │
│                                                                                                                 │
│  1. **Job Displacement vs. Job Creation**                                                                       │
│     - The **World Economic Forum’s “Future of Jobs Report 2023”** estimates that by 2027, AI and automation     │
│  could displace **83 million jobs**, while also creating **69 million new jobs** globally. This indicates a     │
│  net loss in employment overall, which emphasizes the transition that workers would face (Source: [World        │
│  Economic Forum](https://www.weforum.org/publications/the-future-of-jobs-report-2023/)).                        │
│                                                                                                                 │
│  2. **Automation Potential**                                                                                    │
│     - A report by **Goldman Sachs** posits that **300 million full-time jobs** worldwide are at risk from AI    │
│  automation, underscoring generative AI's potential to disrupt a wide range of industries (Source: [Goldman     │
│  Sachs Report](https://www.iedconline.org/clientuploads/EDRP%20Logos/AI_Impact_on_Labor_Markets.pdf)).          │
│                                                                                                                 │
│  3. **Industry-Specific Impacts**                                                                               │
│     - The **Bureau of Labor Statistics (BLS)** anticipates a **17.9%** increase in employment for software      │
│  developers from 2023 to 2033, which greatly outpaces the average growth rate of **4%** across all              │
│  occupations, highlighting the increasing demand for AI-related skills (Source: [BLS                            │
│  Report](https://www.bls.gov/opub/ted/2025/ai-impacts-in-bls-employment-projections.htm)).                      │
│     - A **McKinsey report** indicates that **30% of current U.S. jobs** could be automated by 2030, suggesting  │
│  profound changes in the job landscape due to AI. This indicates a serious understanding of the shifts          │
│  required in the workforce (Source: [Forbes                                                                     │
│  Article](https://www.forbes.com/sites/jackkelly/2025/